In [0]:
!pip install -q tensorflow-gpu==2.0.0-rc1

In [0]:
!pip install tf-nightly-gpu

In [0]:
!pip install import_ipynb

In [0]:
# mount to your google drive
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

In [0]:
#ipynb 모듈 불러오기 위한 설정(model import)
import sys
sys.path.append('/content/gdrive/My Drive')
sys.path

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image

import math
import glob
import os
import numpy as np
import cv2
import pickle

import matplotlib.pyplot as plt
import albumentations as A
import seaborn as sns

import import_ipynb
from model_class_cp import EFFICIENTNET_B0_7, MobileNet_V1, VggNet, ResNet, DenseNet

print(tf.__version__)

## 설정(텐서, 배치, 스텝, 에폭, csv 등.)

In [0]:
#train, test, sample csv 읽어오기
train = pd.read_csv('/content/gdrive/My Drive/cvpr_data/train.csv', index_col=None)
test = pd.read_csv('/content/gdrive/My Drive/cvpr_data/test.csv', index_col=None)
sample = pd.read_csv('/content/gdrive/My Drive/cvpr_data/sample_submission.csv', index_col=None)

#root_path 경로 설정.
ROOT_PATH = '/content/gdrive/My Drive/cvpr_data/images'

# 이미지 설정(train : 1821, test : 1821)
IMAGE_SHAPE = [512, 768, 3]

train_batch = 10
val_batch = 10
test_batch = 5

train_step = 80
val_step = 20
epoch = 100
history = {}

#classes
NUM_CLASSES = 4
CLASSES = ['healthy', 'multiple_diseases', 'rust', 'scab']

## 모델 선택(해당 모델 번호 입력) 

In [0]:
def get_models(model_select):
  if model_select == 1:
    return VggNet.vgg_11(NUM_CLASSES)
  elif model_select == 2:
    return VggNet.vgg_13(NUM_CLASSES)
  elif model_select == 3:
    return VggNet.vgg_16(NUM_CLASSES)
  elif model_select == 4:
    return VggNet.vgg_19(NUM_CLASSES)
  elif model_select == 5:
    return VggNet.se_vgg_16(NUM_CLASSES)
  elif model_select == 6:
    return VggNet.se_vgg_19(NUM_CLASSES)
  elif model_select == 7:
    return ResNet.resnet_18(NUM_CLASSES)
  elif model_select == 8:
    return ResNet.resnet_34(NUM_CLASSES)
  elif model_select == 9:
    return ResNet.resnet_50(NUM_CLASSES)
  elif model_select == 10:
    return ResNet.resnet_101(NUM_CLASSES)
  elif model_select == 11:
    return ResNet.resnet_152(NUM_CLASSES)
  elif model_select == 12:
    return ResNet.se_resnet_50(NUM_CLASSES)
  elif model_select == 13:
    return ResNet.se_resnet_101(NUM_CLASSES)
  elif model_select == 14:
    return ResNet.se_resnet_152(NUM_CLASSES)
  elif model_select == 15:
    return DenseNet.DenseNet_121(NUM_CLASSES)
  elif model_select == 16:
    return DenseNet.DenseNet_169(NUM_CLASSES)
  elif model_select == 17:
    return DenseNet.DenseNet_201(NUM_CLASSES)
  elif model_select == 18:
    return DenseNet.DenseNet_265(NUM_CLASSES)
  elif model_select == 19:
    return DenseNet.se_DenseNet_121(NUM_CLASSES)
  elif model_select == 20:
    return DenseNet.se_DenseNet_169(NUM_CLASSES)
  elif model_select == 21:
    return DenseNet.se_DenseNet_201(NUM_CLASSES)
  elif model_select == 22:
    return DenseNet.se_DenseNet_265(NUM_CLASSES)
  elif model_select == 23:
    return EFFICIENTNET_B0_7.efficient_net_b0(NUM_CLASSES)
  elif model_select == 24:
    return EFFICIENTNET_B0_7.efficient_net_b1(NUM_CLASSES)
  elif model_select == 25:
    return EFFICIENTNET_B0_7.efficient_net_b2(NUM_CLASSES)
  elif model_select == 26:
    return EFFICIENTNET_B0_7.efficient_net_b3(NUM_CLASSES)
  elif model_select == 27:
    return EFFICIENTNET_B0_7.efficient_net_b4(NUM_CLASSES)
  elif model_select == 28:
    return EFFICIENTNET_B0_7.efficient_net_b5(NUM_CLASSES)
  elif model_select == 29:
    return EFFICIENTNET_B0_7.efficient_net_b6(NUM_CLASSES)
  elif model_select == 30:
    return EFFICIENTNET_B0_7.efficient_net_b7(NUM_CLASSES)
  elif model_select == 31:
    return MobileNet_V1.MobileNet_V1(NUM_CLASSES)
  elif model_select == 32:
    return MobileNet_V1.se_MobileNet_V1(NUM_CLASSES)
  else:
    raise ValueError("The model_index does not exist.")

In [0]:
model_select = int(input('Please enter the model index you want to use.'))

Please enter the model index you want to use.3


In [0]:
get_model = get_models(model_select)

## Dataset Directory & Data loader Implementation  

In [0]:
#albumentations Module 설정.
transforms_train = A.Compose([
    A.Resize(height=IMAGE_SHAPE[0], width=IMAGE_SHAPE[1], p=1.0),
    A.RandomCrop(height=IMAGE_SHAPE[0]-2, width=IMAGE_SHAPE[1]-2, p=1.0),
    A.Resize(height=IMAGE_SHAPE[0], width=IMAGE_SHAPE[1], p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=40.0, p=0.8),
    A.HorizontalFlip(p=0.5)
])

transforms_valid = A.Compose([
    A.Resize(height=IMAGE_SHAPE[0], width=IMAGE_SHAPE[1], p=1.0)
])

In [0]:
images = []
test_images = []

image_ids = list(train.image_id)
test_ids = list(test.image_id)

for image_id in image_ids:
  images.append(os.path.join(ROOT_PATH,image_id) + '.jpg')
for test_id in test_ids:
  test_images.append(os.path.join(ROOT_PATH, test_id) + '.jpg')

labels = list(zip(train.healthy, train.multiple_diseases, train.rust, train.scab))
test_labels = []

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size = 0.2, random_state=2020, stratify=labels)

In [0]:
print(len(train_images), len(val_images), len(train_labels), len(val_images))

In [0]:
train_images[0], val_images[0]

In [0]:
img = plt.imread(train_images[0])
print(img.shape)
plt.imshow(img)

img = plt.imread(val_images[0])
print(img.shape)
plt.imshow(img)

In [0]:
# 데이터 확인 함수
def check_images(labels, classes):
  healthy = labels.healthy
  multiple_diseases = labels.multiple_diseases
  rust =labels.rust
  scab= labels.scab

  num_healthy = [len([i for i in healthy if i == 1]), len([i for i in healthy if i == 0])]
  num_multiple = [len([i for i in multiple_diseases if i == 1]), len([i for i in multiple_diseases if i == 0])]
  num_rust = [len([i for i in rust if i == 1]), len([i for i in rust if i == 0])]
  num_scab = [len([i for i in scab if i == 1]), len([i for i in scab if i == 0])]

  pos = [num_healthy[0], num_multiple[0], num_rust[0], num_scab[0]]
  nev = [num_healthy[1], num_multiple[1], num_rust[1], num_scab[1]]

  plt.rcParams["font.size"] = 12

  plt.figure(figsize=(12,8))

  x = np.arange(len(classes))
  p1 = plt.bar(x-0.15, pos, width=0.3, color='#FF0000', label = 1, alpha=0.5)
  plt.xticks(x, classes)
  p2 = plt.bar(x+0.15, nev, width=0.3, color='#0000FF', label = 0, alpha = 0.5)
  plt.xticks(x, classes)

  for i, rect in enumerate(p1):
      plt.text(rect.get_x() + rect.get_width() / 2.0, 0.95 * rect.get_height(), str(pos[i]), ha='center')
  for i, rect in enumerate(p2):
      plt.text(rect.get_x() + rect.get_width() / 2.0, 0.95 * rect.get_height(), str(nev[i]), ha='center')
  plt.legend((p1[0], p2[0]), ('1','0'), fontsize = 15)

  plt.show()

In [0]:
check_images(train, CLASSES)

In [0]:
class Classification_Dataset:
    # class_list -> image_labels 변경
    def __init__(self, image_paths = "", image_labels = "", target_size = None, augment = None):
        
        self.image_list = image_paths
        self.image_labels = image_labels
        self.target_size = target_size
        self.augment = augment
        #self.num_of_class = len(self.class_list)
    
    
    def __len__(self):
        return len(self.image_list)

    
    def __getitem__(self, idx):
        image = cv2.imread(self.image_list[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image/255.0
        label = self.image_labels[idx]
        
        #for i in range(self.num_of_class):
        #    if(self.class_list[i] in self.image_list[idx]):
        #        label = i
        #        break             

        
        if self.augment:
            transformed = self.augment(image=image)
            image = transformed['image']

        #image = image.resize(self.target_size)
        return image, label

In [0]:
class Classification_Data_Loader(Sequence):

    def __init__(self, dataset, batch_size=None, shuffle=False):
        
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle=shuffle
        self.index_list = [idx for idx in range(len(self.dataset))]
        self.idx=0
        
    def __getitem__(self, idx):
      
        start = idx * self.batch_size
        end = (idx+1) * self.batch_size
        data = []
        label = []
        
        if self.shuffle:
            np.random.shuffle(self.index_list)
            
        for j in range(start,end):
            if j >= len(self.index_list):
                j%=len(self.dataset)
            data.append(self.dataset[self.index_list[j]])
      
        batch = tuple(tf.stack(sample, axis=0) for sample in zip(*data))

        if self.idx >= (len(self.dataset)//self.batch_size):
            self.idx=0
        self.idx +=1
        return batch

    def __call__(self):
        batch = self.__getitem__(self.idx)
        return batch

    def __len__(self):
        return (len(self.dataset) // self.batch_size)

    def get_batch(self):
        return self.batch_size

In [0]:
# 데이터 로더 동작 확인
def data_info_print(data_gen, idx):
  batch = data_gen.__getitem__(idx)
  print(type(batch), type(batch[0]), type(batch[1]))
  # 변경
  print(len(batch[0]), len(batch[1]))
  plt.figure(figsize=(20,8))
  for i in range(len(batch[0])):
    plt.subplot(2,5,i+1)
    # 변경 
    plt.imshow(batch[0][i])
    
  print('label :', batch[1])
  print(type(batch[0][0]), type(batch[1][0]))

# 데이터 확인 & 데이터 로더 생성



In [0]:
train_dataset = Classification_Dataset(image_paths=train_images, image_labels = train_labels, augment = transforms_train)
valid_dataset = Classification_Dataset(image_paths=val_images, image_labels = val_labels, augment = transforms_valid)

train_data_gen = Classification_Data_Loader(dataset=train_dataset, batch_size=train_batch, shuffle=True)
valid_data_gen = Classification_Data_Loader(dataset=valid_dataset, batch_size=val_batch, shuffle=True)

images, labels = train_data_gen.__getitem__(0)
print(labels)

In [0]:
data_info_print(train_data_gen,0)

# 모델 훈련  
### Using GradientTape


In [0]:
def fit_test(model, train_gen, train_steps, epochs, val_gen, val_steps):
    BATCH_SIZE = train_gen.get_batch()
  # define loss and optimizer
    optimizer = keras.optimizers.Adam(learning_rate=0.0000779)
    loss_func = keras.losses.CategoricalCrossentropy()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

    valid_loss = tf.keras.metrics.Mean(name='valid_loss')
    valid_accuracy = tf.keras.metrics.CategoricalAccuracy(name='valid_accuracy')


    train_losses = []
    train_acces = []
    val_losses = []
    val_acces = []


    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = loss_func(y_true=labels, y_pred=predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

    @tf.function
    def valid_step(images, labels):

        predictions = model(images, training=False)
        v_loss = loss_func(labels, predictions)

        valid_loss(v_loss)
        valid_accuracy(labels, predictions)

    def step_decay(epoch, k):
        if epoch < k :
          return True
        #연속으로 k-1번 떨어지면 learning rate 조정.
        F_len = 0

        for step_k in range(k, 1, -1):
          step_k = -step_k
          if val_losses[step_k + 1] > val_losses[step_k]:
            #print('k+1 : ', val_losses[step_k + 1], ' > ', 'k : ', val_losses[step_k])
            F_len = F_len + 1

        if F_len == k-1:
          if 1e-06 >= optimizer.learning_rate:
            optimizer.learning_rate = 1e-06
          else:
            optimizer.learning_rate = optimizer.learning_rate - (optimizer.learning_rate / 5)
          
          print('step_decay is reduce %6.7f.' % (optimizer.learning_rate))

  # start training
    for epoch in range(epochs):
        step_decay(epoch, 4)
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        valid_loss.reset_states()
        valid_accuracy.reset_states()
        #step = 0
        for step in range(train_steps):
            print('.',end='')
            if step % 100 == 0:
                print()
            images, labels = train_gen()
            train_step(images, labels)
            
        print('x')
        
        for val_step in range(val_steps):
            print('.', end='')
            if val_step % 100 == 0:
                print()
            valid_images, valid_labels = val_gen()
            valid_step(valid_images, valid_labels)
        print('x')
        print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
              "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                epochs,
                                                                train_loss.result(),
                                                                train_accuracy.result(),
                                                                valid_loss.result(),
                                                                valid_accuracy.result()))
        train_losses.append(train_loss.result())
        train_acces.append(train_accuracy.result())
        val_losses.append(valid_loss.result())
        val_acces.append(valid_accuracy.result())

    history = {'train_losses': train_losses, 'train_acces': train_acces, 'val_losses': val_losses, 'val_acces': val_acces}
    return history

In [0]:
 def test_eval(model, test_data_gen, test_steps):
  loss_func = keras.losses.BinaryCrossentropy()
  test_loss = tf.keras.metrics.Mean()
  test_accuracy = tf.keras.metrics.BinaryAccuracy()

  @tf.function
  def test_step(images, labels):
      predictions = model(images, training=False)
      t_loss = loss_func(labels, predictions)  

      test_loss(t_loss)
      test_accuracy(labels, predictions)

  for step in range(test_steps):
    print('.',end='')
    if step % 100 == 0:
      print()
    test_images, test_labels = test_data_gen()
    test_step(test_images, test_labels)
  print('x')
  print("loss: {:.5f}, test accuracy: {:.5f}".format(test_loss.result(),
                                                      test_accuracy.result()))

  #print("The accuracy on test set is: {:.3f}%".format(test_accuracy.result()*100))

In [0]:
get_model.build(input_shape=(None, IMAGE_SHAPE[0], IMAGE_SHAPE[1], IMAGE_SHAPE[2]))

In [0]:
get_model.summary()

In [0]:
history = fit_test(model=get_model, 
                   train_gen=train_data_gen, 
                   train_steps=train_step, 
                   epochs=epoch, 
                   val_gen=valid_data_gen, 
                   val_steps=val_step)

# 저장

In [0]:
get_model.save_weights('/content/gdrive/My Drive/get_model', save_format='tf')

In [0]:
with open('/content/gdrive/My Drive/junsick_coding/model_history.bin', 'wb') as hi:
  pickle.dump(history, hi)

## 로드

In [0]:
get_model.load_weights('/content/gdrive/My Drive/get_model')   

In [0]:
with open('/content/gdrive/My Drive/junsick_coding/model_history.bin', 'rb') as f:
  re_history = pickle.load(f)

# 모델 학습 가시화 그래프


In [0]:
def his_graph(history, epoch):
  acc = history['train_acces']
  val_acc = history['val_acces']

  loss = history['train_losses']
  val_loss = history['val_losses']

  epochs_range = range(epoch)

  plt.figure(figsize=(8, 8))
  plt.subplot(2, 1, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(2, 1, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')

  plt.show()

In [0]:
his_graph(history, epoch)

#모델 예측 및 가시화(confusion matrix)

In [0]:
def model_predict(model, test_gen, steps):
  predics=[]
  labels = []
  for step in range(steps):
    image,label = test_gen()
    pred = model(image)
    labels.append(label)
    predics.append(pred)
  re_dic = dict({'pred':predics, 'labels':labels})
  return re_dic

In [0]:
pred_dataset = Dataset_dir_ver(path=test_PATH)
pred_data_gen = Data_Loader(dataset=test_dataset, batch_size=1)

In [0]:
def pred_confusion_matrix(pred, labels):
  pred_int = []
  for i in range(len(pred)):
    if pred[i] > 0.5:
      pred_int.append(1)
    elif pred[i] <= 0.5:
      pred_int.append(0)

  print(len(pred))
  print(len(pred_int))
  print('Confusion Matrix')
  conf_matrix = confusion_matrix(labels, pred_int)
  print(conf_matrix)
  sns.heatmap(conf_matrix,cmap="Blues",annot=True,fmt='g');
  plt.xlabel('predicted value')
  plt.ylabel('true value');

In [0]:
re_dic = model_predict(get_model, pred_data_gen, 4000)
pred = re_dic['pred']
labels = re_dic['labels']
pred = list(map(float,pred))

In [0]:
pred_confusion_matrix(pred, labels)

#Grad CAM
######알맞게 모델 조정 필요(return output, mask)

In [0]:
def grad_cam(model, idx, class_name):
  plt.figure(figsize=(18, 15))
  for i in range(40,50):
    path = os.path.join('/content/gdrive/My Drive/cvpr_data/images/Train_%d.jpg'%i)
    image = cv2.imread(path)
    img = cv2.resize(image, (IMAGE_SHAPE[0], IMAGE_SHAPE[1]))
    x = img.copy()
    x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    
    #grad_model = tf.keras.models.Model(
    #      [resnet101.layers[0]], [resnet101.layers[-2],resnet101.layers[-1]]
    #  )
      
    with tf.GradientTape() as tape:
        inputs = tf.cast(x, tf.float32)
        model_outputs, predictions = model(inputs, cam = 'grad')
        loss = predictions[:,0]

    grads = tape.gradient(loss, model_outputs)

    guided_grads = (
          tf.cast(model_outputs > 0, "float32") * tf.cast(grads > 0, "float32") * grads
    )

    prediction = predictions[0][idx]
    model_outputs = model_outputs[0]
    plt.subplot(4, 5, i-40+1)
    weights = np.mean(grads, axis=(1, 2))
    weights = weights.reshape(2048, 1)

    cam = (prediction -0.5) * np.matmul(model_outputs, weights)
    cam -= np.min(cam)
    cam /= np.max(cam)
    cam -= 0.2
    cam /= 0.8

    try:
      cam = cv2.resize(np.float32(cam), (IMAGE_SHAPE[0], IMAGE_SHAPE[1]))
    except Exception as e:
      #print(cam.shape)
      print(str(e))

    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    heatmap[np.where(cam <= 0.2)] = 0
    grad_cam = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)
    plt.axis('off')
    plt.title('%s_class_grad_cam'%class_name)
    plt.imshow(grad_cam[:, :, ::-1])